In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import json

In [42]:
#rename the column names to match the ones in the json file

with open("Datasets/WebNLG/57_triples/test_57triples.json", "r") as jsonFile:
    data = json.load(jsonFile)

for i in range(len(data)):
    data[i]["Types_KG"] = data[i].pop("Types KG")
    data[i]["Instances_KG"] = data[i].pop("Instances KG")
    data[i]["Subclasses_KG"] = data[i].pop("Subclasses KG")


# with open("Datasets/WebNLG/57_triples/train_57.json", "w") as jsonFile:
#     json.dump(data, jsonFile)

In [10]:
#select from test set only stories with more than 5 triples 

with open("Datasets/WebNLG/57_triples/test_57.json", "r") as jsonFile:
    data = json.load(jsonFile)
new_list = []
for i in data:
    x = i['Instances_KG'].count('|')
    if x >= 10:
        #print(i)
        new_list.append(i)

data = new_list
import json 
with open('Datasets/WebNLG/57_triples/test_57_1.json', 'w') as f:
    f.write('[')
    for i in data:
        json.dump(i, f)
        f.write(',')
        f.write('\n')
    f.write('{}')    
    f.write(']')    


In [43]:
import json 
with open('Datasets/WebNLG/test_57.json', 'w') as f:
    f.write('[')
    for i in data:
        json.dump(i, f)
        f.write(',')
        f.write('\n')
    f.write('{}')    
    f.write(']')

In [2]:
#merge xml files in one file

import os
import xml.etree.ElementTree as ET

# Path to the directory containing the XML files
xml_dir = "WebNLG/release_v3.0/en/dev/7triples/"

# Initialize the root element for the merged XML file
merged_root = ET.Element("root")

# Loop through all XML files in the directory
for filename in os.listdir(xml_dir):
    if filename.endswith(".xml"):
        # Parse the XML file into an ElementTree object
        tree = ET.parse(os.path.join(xml_dir, filename))

        # Get the root element of the parsed XML
        root = tree.getroot()

        # Append the root element to the merged XML file
        merged_root.extend(root)

# Create an ElementTree object for the merged XML
merged_tree = ET.ElementTree(merged_root)

# Write the merged XML to a file
merged_tree.write("WebNLG/release_v3.0/en/dev/7triples/dev_7triples.xml")


In [233]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
def get_entity_class(entity, subclass=False,multiple=False):
    #construct the SPARQL query to retrieve the class of the entity


    query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?class ?label WHERE {
        <%s> rdf:type ?class .
        ?class rdfs:subClassOf* owl:Thing .
        ?class rdfs:label ?label .
        FILTER (lang(?label) = "en")
    }
    """ % entity 


    # set the SPARQL endpoint and query string
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # execute the query and retrieve the results
    results = sparql.query().convert()
    bindings = results['results']['bindings']
    if multiple:
        
        class_uri = []
        class_label = []
        for i in range(len(bindings)):
            class_uri.append(bindings[i]['class']['value'])
            class_label.append(bindings[i]['label']['value'])
        if class_uri != None:
            
            if subclass:
                return list(set(class_uri))[:3]
            else:
                return list(set(class_label))[:3]
    
    # extract the class label from the results
    if len(bindings) > 0 and bindings != None:

        class_uri = bindings[0]['class']['value']
        class_label = bindings[0]['label']['value']
        if class_uri != None:
            if subclass:
                return class_uri
            else:
                return class_label

    

def get_entity_subclass(entity):
    clas = entity
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    # construct the SPARQL query to retrieve the class of the entity
    query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?class ?label WHERE {
        <%s> rdfs:subClassOf ?class .

        ?class rdfs:label ?label .
        FILTER (lang(?label) = "en")

    }
    """ % clas

    # set the SPARQL endpoint and query string
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # execute the query and retrieve the results
    results = sparql.query().convert()
    bindings = results['results']['bindings']

    # extract the class label from the results
    if len(bindings) > 0:
        class_uri = bindings[0]['class']['value']
        class_label = bindings[0]['label']['value']
        return class_label
    else:
        return None  
    

In [213]:
get_entity_class('http://dbpedia.org/resource/Brazil', subclass=True, multiple = True)
#get_entity_subclass('http://dbpedia.org/ontology/PopulatedPlace')

['http://dbpedia.org/ontology/Place',
 'http://dbpedia.org/ontology/Country',
 'http://dbpedia.org/ontology/PopulatedPlace']

In [300]:
with open("Datasets/WebNLG/57_triples/oneClass/space_triples/test_57_oneClass.json", "r") as jsonFile:
    data = json.load(jsonFile)

#' - '.join(set([f"{entity.replace(' ', '_')} | type | {get_entity_class('http://dbpedia.org/resource/'+entity)}" for entity in entities if '"' not in entity]))    

In [301]:
for i in range(len(data)):
    data[i]['Instances_KG'] = str(data[i]['Instances_KG']).replace('_',' ')

KeyError: 'Instances_KG'

In [295]:
#spazio
for i in range(len(data)):
    data[i]['Instances_KG'] = str(data[i]['Instances_KG']).replace(' - ',' ')
    data[i]['Types_KG'] = data[i]['Types_KG'].replace(' - ',' ')
    data[i]['Subclasses_KG'] = data[i]['Subclasses_KG'].replace(' - ',' ')

KeyError: 'Instances_KG'

In [289]:
data

[{'story': 'Alan Bean was born in Wheeler, Texas and is from the U.S. He graduated in 1955 from UT Austin with a B.S. He worked as a test pilot and for NASA in 1963. He spent 100305 minutes in space and is now retired.',
  'Types_KG': 'Alan Bean type person | Test pilot type person | United States type person | Wheeler, Texas type place | 1963 type time period',
  'Instances_KG': 'Alan Bean nationality United States | Alan Bean occupation Test pilot | Alan Bean almaMater "UT Austin, B.S. 1955"@en | Alan Bean birthPlace Wheeler, Texas | Alan Bean timeInSpace "100305.0"^^<http://dbpedia.org/datatype/minute> | Alan Bean selection 1963 | Alan Bean status "Retired"@en',
  'Subclasses_KG': 'place subclass Thing | person subclass animal | time period subclass Thing'},
 {'story': 'Alan Bean is a US national, who was born in Wheeler, Texas. He received a B.S. degree from UT at Austin in 1955. He performed as a test pilot. He was selected by NASA in 1963 and became a crew member of Apollo 12. He

In [285]:
#trattini
for i in range(len(data)):
    data[i]['Instances_KG'] = str(data[i]['Instances_KG']).replace(' - ','|').replace(' | ','-')
    data[i]['Instances_KG'] = str(data[i]['Instances_KG']).replace('-',' - ').replace('|',' | ')
    data[i]['Types_KG'] = data[i]['Types_KG'].replace(' - ','|').replace(' | ','-')
    data[i]['Types_KG'] = data[i]['Types_KG'].replace('-',' - ').replace('|',' | ')
    data[i]['Subclasses_KG'] = data[i]['Subclasses_KG'].replace(' - ','|').replace(' | ','-')
    data[i]['Subclasses_KG'] = data[i]['Subclasses_KG'].replace('-',' - ').replace('|',' | ')



 

KeyError: 'Instances_KG'

In [302]:
with open("Datasets/WebNLG/57_triples/oneClass/space_triples/test_57_oneClass.json", 'w') as f:
    f.write('[')
    for i in data:
        json.dump(i, f)
        f.write(',')
        f.write('\n')
    f.write('{}')    
    f.write(']')


In [276]:
trattini("Datasets/WebNLG/57_triples/oneClass/test_57_oneClass.json", "Datasets/WebNLG/57_triples/oneClass/test_57_oneClass.json")


KeyError: 'Instances_KG'

In [237]:
from tqdm import tqdm
#def get_multiple_class(data):
for i in tqdm(range(len(data))):
    #entities
    entities = [triple.split(' | ')[2] for triple in data[i]['Instances_KG'].split(' - ')]
    entities.append(str(data[i]['Instances_KG'].split(' | ')[0]))
    entities = [entity.replace(' ', '_') for entity in entities]
    #print(entities)
    #print([('http://dbpedia.org/resource/'+entity) for entity in entities])

    #classes
    classes = [get_entity_class('http://dbpedia.org/resource/'+entity,multiple=True)for entity in entities if '"' not in entity ]
    classes = list(filter(lambda item: item is not None, classes))
    #print(classes)
    #classes uri
    classes_uri = [get_entity_class('http://dbpedia.org/resource/'+entity, subclass=True, multiple=True)for entity in entities if '"' not in entity ]
    classes_uri = list(filter(lambda item: item is not None, classes_uri))


    #KG
    data[i]['Types_KG'] = ' - '.join(set([f"{entity.replace('_', ' ')} | type | {get_entity_class('http://dbpedia.org/resource/'+entity, multiple=True)[i]}" for entity in entities if '"' not in entity if get_entity_class('http://dbpedia.org/resource/'+entity) != None for i in range(len(get_entity_class('http://dbpedia.org/resource/'+entity, multiple=True)))]))
    #data[i]['Types_KG'] = ' - '.join(set([f"{entity} | type | {c}" for entity,c in zip(entities,classes) if '"' not in entity ]))
    print(data[i]['Types_KG'])
    
    data[i]['Subclasses_KG'] =  ' - '.join(set([f"{s} | subclass | {get_entity_subclass(t)}" for i,j in zip(classes, classes_uri) for s,t in zip(i,j)]))
    print(data[i]['Subclasses_KG'])
    data[i]['Instances_KG'] = data[i]['Instances_KG'].replace('_', ' ')
    print(data[i]['Instances_KG'])

#get_multiple_class(data)

    # with open('Datasets/WebNLG/57_triples/test_57_oneClass.json', 'w') as f:
    #     f.write('[')
    #     for i in data:
    #         json.dump(i, f)
    #         f.write(',')
    #         f.write('\n')
    #     f.write('{}')    
    #     f.write(']')


  0%|          | 0/342 [00:00<?, ?it/s]

Brazil | type | populated place - Arapiraca | type | place - Estádio Municipal Coaracy da Mata Fonseca | type | venue - Arapiraca | type | settlement - Brazil | type | place - Estádio Municipal Coaracy da Mata Fonseca | type | building - Arapiraca | type | populated place - Brazil | type | country - Estádio Municipal Coaracy da Mata Fonseca | type | architectural structure - Campeonato Brasileiro Série C | type | soccer league


  0%|          | 1/342 [00:04<27:51,  4.90s/it]

soccer league | subclass | sports league - place | subclass | None - place | subclass | populated place - settlement | subclass | settlement - populated place | subclass | populated place - building | subclass | architectural structure - venue | subclass | None - populated place | subclass | place - country | subclass | None - architectural structure | subclass | building
Estádio Municipal Coaracy da Mata Fonseca | location | Arapiraca - Agremiação Sportiva Arapiraquense | league | Campeonato Brasileiro Série C - Campeonato Brasileiro Série C | country | Brazil - Agremiação Sportiva Arapiraquense | nickname | "''Alvinegro"@en - Agremiação Sportiva Arapiraquense | ground | Estádio Municipal Coaracy da Mata Fonseca
FC Spartak Moscow | type | agent - Otkrytiye Arena | type | stadium - Aleksandr Prudnikov | type | animal - Aleksandr Prudnikov | type | person - FC Spartak Moscow | type | organisation - Aleksandr Prudnikov | type | eukaryote - FC Spartak Moscow | type | sports club


  1%|          | 2/342 [00:07<19:17,  3.40s/it]

...

  1%|          | 3/342 [00:09<15:04,  2.67s/it]

place | subclass | None - settlement | subclass | populated place - city | subclass | populated place - administrative region | subclass | region - populated place | subclass | settlement - person function | subclass | None
Ciudad Ayala | populationMetro | 1777539 - Ciudad Ayala | leaderTitle | City Manager - Ciudad Ayala | type | City - Ciudad Ayala | populationDensity | 1604.0 - Ciudad Ayala | governmentType | Council-manager government
Mexico | type | person - Ciudad Ayala | type | city - Ciudad Ayala | type | place - Mexico | type | country - Ciudad Ayala | type | populated place - Mexico | type | place


  1%|          | 4/342 [00:11<16:00,  2.84s/it]

country | subclass | None - place | subclass | None - city | subclass | populated place - person | subclass | animal - populated place | subclass | settlement - place | subclass | populated place
Ciudad Ayala | populationDensity | 1604.0 - Ciudad Ayala | leaderTitle | Governator - Ciudad Ayala | country | Mexico - Ciudad Ayala | elevation | 1147.0 - Ciudad Ayala | timeZone | Pacific Daylight Time


KeyboardInterrupt: 

In [89]:
entities = [triple.split(' | ')[2] for triple in data[8]['Instances_KG'].split(' - ')]
entities.append(str(data[8]['Instances_KG'].split(' | ')[0]))
entities = list(set(entities))
print(entities)
print(data[8]['Instances_KG'])
classes = [get_entity_class('http://dbpedia.org/resource/'+entity)for entity in entities if '"' not in entity ]
classes = list(set(list(filter(lambda item: item is not None, classes))))
classes_uri = [get_entity_class('http://dbpedia.org/resource/'+entity, subclass=True)for entity in entities if '"' not in entity ]
classes_uri = list(set(list(filter(lambda item: item is not None, classes_uri))))
print(classes_uri)
print(' - '.join(set([f"{entity.replace(' ', '_')} | type | {get_entity_class('http://dbpedia.org/resource/'+entity)}" for entity in entities if '"' not in entity]))) #if get_entity_class('http://dbpedia.org/resource/'+entity)!= None])) )
print(' - '.join(set([f"{e.replace(' ', '_')} | type | {c}" for e,c in zip(entities,classes)])))
print( ' - '.join(set([f"{i} | subclass | {get_entity_subclass(j)}" for i,j in zip(classes, classes_uri)])))

['George_Allen_&_Unwin', 'The_Fellowship_of_the_Ring', 'The_Two_Towers', 'J._R._R._Tolkien', 'Fantasy_(genre)', '1954-07-29']
The_Fellowship_of_the_Ring | publisher | George_Allen_&_Unwin - The_Fellowship_of_the_Ring | literaryGenre | Fantasy_(genre) - The_Fellowship_of_the_Ring | subsequentWork | The_Two_Towers - The_Fellowship_of_the_Ring | author | J._R._R._Tolkien - The_Fellowship_of_the_Ring | releaseDate | 1954-07-29
['http://dbpedia.org/ontology/Work', 'http://dbpedia.org/ontology/Publisher', 'http://dbpedia.org/ontology/Person', 'http://dbpedia.org/ontology/MusicGenre']
George_Allen_&_Unwin | type | publisher - Fantasy_(genre) | type | music genre - The_Fellowship_of_the_Ring | type | work - 1954-07-29 | type | None - J._R._R._Tolkien | type | person - The_Two_Towers | type | work
The_Fellowship_of_the_Ring | type | person - J._R._R._Tolkien | type | publisher - George_Allen_&_Unwin | type | music genre - The_Two_Towers | type | work
http://dbpedia.org/ontology/Work
http://dbpe

In [21]:
import xml.etree.ElementTree as ET
import xml.etree.ElementTree as ET
from SPARQLWrapper import SPARQLWrapper, JSON

# set up the SPARQL endpoint for DBpedia
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

# load the WebNLG dataset from the XML file
#tree = ET.parse("WebNLG/release_v3.0/en/dev/7triples/dev_7triples.xml")
tree = ET.parse("WebNLG/release_v3.0/en/test/rdf-to-text-generation-test-data-with-refs-en.xml")
#tree = ET.parse("WebNLG/release_v3.0/en/train/train_57triples.xml")
root = tree.getroot()

def get_entity_class(entity, subclass=False):
    # construct the SPARQL query to retrieve the class of the entity
    query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?class ?label WHERE {
        <%s> rdf:type ?class .
        ?class rdfs:subClassOf* owl:Thing .
        ?class rdfs:label ?label .
        FILTER (lang(?label) = "en")
    }
    """ % entity

    # set the SPARQL endpoint and query string
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # execute the query and retrieve the results
    results = sparql.query().convert()
    bindings = results['results']['bindings']

    # extract the class label from the results
    if len(bindings) > 0:
        class_uri = bindings[0]['class']['value']
        class_label = bindings[0]['label']['value']
        if subclass:
            return class_uri
        else:
            return class_label
    else:
        return None

def get_entity_subclass(entity):
    clas = get_entity_class(entity, subclass = True)
    print(clas)
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    # construct the SPARQL query to retrieve the class of the entity
    query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?class ?label WHERE {
        <%s> rdfs:subClassOf ?class .
        ?class rdfs:subClassOf* owl:Thing .
        ?class rdfs:label ?label .
        FILTER (lang(?label) = "en")
    }
    """ % clas

    # set the SPARQL endpoint and query string
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # execute the query and retrieve the results
    results = sparql.query().convert()
    bindings = results['results']['bindings']

    # extract the class label from the results
    if len(bindings) > 0:
        class_uri = bindings[0]['class']['value']
        class_label = bindings[0]['label']['value']
        return class_label
    else:
        return None    


# Initialize an empty list to store the results
results = []

# Iterate over all entry elements
for entry in tree.findall('.//entry'):

    # Initialize an empty dictionary for this entry
    entry_dict = {}

    # Extract the eid attribute and store it as the entry ID
    #entry_dict['Entry ID'] = entry.get('eid')

    # Initialize an empty list to store the otriples for this entry
    otriples = []

    # Iterate over all otriple elements in this entry's originaltripleset
    for otriple in entry.findall('.//originaltripleset/otriple'):

        # Extract the text content of the otriple element and append it to the otriples list
        otriples.append(otriple.text)
    print(otriples)

    # Join the otriples list into a single string, with each triple separated by '-'
    
    entry_dict['story'] = entry.find('.//lex').text
    entry_dict['Instances KG'] = ' - '.join(otriples)
    entities = [triple.split(' | ')[2] for triple in otriples]
    entities.append(str(otriples[0].split(' | ')[0]))
    print(entities)
    entities = [entity.replace(" ", "_") for entity in entities]

    # a = [get_entity_class('http://dbpedia.org/resource/'+entity) for entity in entities if '"' not in entity]
    entry_dict['Types KG'] =  ' - '.join(set([f"{entity.replace(' ', '_')} | type | {get_entity_class('http://dbpedia.org/resource/'+entity)}" for entity in entities if '"' not in entity]))
    print(entry_dict['Types KG'])
    #entry_dict['Subclasses KG'] =  ' - '.join([f"{entity.replace(' ', '_')} | subclass | {get_entity_subclass('http://dbpedia.org/resource/'+entity)}" for entity in entities if '"' not in entity])
    entry_dict['Subclasses KG'] =  ' - '.join(set([f"{get_entity_class('http://dbpedia.org/resource/'+entity)} | subclass | {get_entity_subclass('http://dbpedia.org/resource/'+entity)}" for entity in entities if '"' not in entity]))
    print(entry_dict['Subclasses KG'])
    

    # Append the entry dictionary to the results list
    results.append(entry_dict)

# Print the results
print(results[0])


['Estádio_Municipal_Coaracy_da_Mata_Fonseca | location | Arapiraca', 'Agremiação_Sportiva_Arapiraquense | league | Campeonato_Brasileiro_Série_C', 'Campeonato_Brasileiro_Série_C | country | Brazil', 'Agremiação_Sportiva_Arapiraquense | nickname | "\'\'Alvinegro"@en', 'Agremiação_Sportiva_Arapiraquense | ground | Estádio_Municipal_Coaracy_da_Mata_Fonseca']
['Arapiraca', 'Campeonato_Brasileiro_Série_C', 'Brazil', '"\'\'Alvinegro"@en', 'Estádio_Municipal_Coaracy_da_Mata_Fonseca', 'Estádio_Municipal_Coaracy_da_Mata_Fonseca']
Arapiraca | type | place - Campeonato_Brasileiro_Série_C | type | soccer league - Brazil | type | place - Estádio_Municipal_Coaracy_da_Mata_Fonseca | type | stadium
http://dbpedia.org/ontology/Place
http://dbpedia.org/ontology/SoccerLeague
http://dbpedia.org/ontology/Place
http://dbpedia.org/ontology/Stadium
http://dbpedia.org/ontology/Stadium
place | subclass | None - soccer league | subclass | sports league - stadium | subclass | venue
['Nie_Haisheng | birthDate | 19

KeyboardInterrupt: 

In [22]:
import json 
with open('Datasets/WebNLG/test_57triples.json', 'w') as f:
    f.write('[')
    for i in results:
        json.dump(i, f)
        f.write(',')
        f.write('\n')
    f.write('{}')    
    f.write(']')


In [37]:
import xml.etree.ElementTree as ET
from SPARQLWrapper import SPARQLWrapper, JSON

# set up the SPARQL endpoint for DBpedia
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

# load the WebNLG dataset from the XML file
tree = ET.parse("WebNLG/release_v3.0/en/train/7triples/Astronaut.xml")
root = tree.getroot()

# define the function to query the class of an entity
def get_entity_class(entity):
    # construct the SPARQL query to retrieve the class of the entity
    query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?class ?label WHERE {
        <%s> rdf:type ?class .
        ?class rdfs:subClassOf* owl:Thing .
        ?class rdfs:label ?label .
        FILTER (lang(?label) = "en")
    }
    """ % entity

    # set the SPARQL endpoint and query string
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # execute the query and retrieve the results
    results = sparql.query().convert()
    bindings = results['results']['bindings']

    # extract the class label from the results
    if len(bindings) > 0:
        class_uri = bindings[0]['class']['value']
        class_label = bindings[0]['label']['value']
        return class_label
    else:
        return None

# loop over all the entries in the dataset
for entry in root.findall(".//entry"):
    # retrieve the triples of the entry
    triples = []
    for otriple in entry.find(".//originaltripleset").findall("otriple"):
        subject, predicate, obj = otriple.text.split(" | ")
        triples.append((subject.strip(), predicate.strip(), obj.strip()))

    # extract the entities from the triples
    entities = set()
    for triple in triples:
        entities.add(triple[0])
        entities.add(triple[2])
    print(entities)    

    # retrieve the class of each entity
    # for entity in entities:
    #     if '"' not in entity:
    #         entity_class = get_entity_class('http://dbpedia.org/resource/'+entity)
    #         print(f'{entity} | type | {entity_class}')

[f"{entity} | type | {get_entity_class('http://dbpedia.org/resource/'+entity)}" for entity in entities if '"' not in entity]


{'United_States', 'Alan_Bean', '"100305.0"^^<http://dbpedia.org/datatype/minute>', '"UT Austin, B.S. 1955"@en', '1963', 'Test_pilot', '"Retired"@en', 'Wheeler,_Texas'}
{'United_States', 'Alan_Bean', '"UT Austin, B.S. 1955"@en', '1963', 'Test_pilot', '"June 1981"@en', 'Apollo_12', 'Wheeler,_Texas'}
{'United_States', 'Alan_Bean', '"100305.0"^^<http://dbpedia.org/datatype/minute>', '"UT Austin, B.S. 1955"@en', 'Test_pilot', '"Retired"@en', 'Apollo_12', 'Wheeler,_Texas'}
{'United_States', 'Alan_Bean', '"100305.0"^^<http://dbpedia.org/datatype/minute>', '"UT Austin, B.S. 1955"@en', '1963', 'Test_pilot', 'Apollo_12', 'Wheeler,_Texas'}
{'United_States', 'Alan_Bean', '"UT Austin, B.S. 1955"@en', 'Test_pilot', '"June 1981"@en', 'Apollo_12', '"1932-03-15"^^xsd:date', 'Wheeler,_Texas'}
{'United_States', 'Alan_Bean', '"UT Austin, B.S. 1955"@en', '1963', 'Test_pilot', 'Apollo_12', '"1932-03-15"^^xsd:date', 'Wheeler,_Texas'}
{'United_States', 'Alan_Bean', '"100305.0"^^<http://dbpedia.org/datatype/mi

['British_Hong_Kong | type | place',
 'United_States | type | person',
 'William_Anders | type | person',
 '1963 | type | time period',
 'Fighter_pilot | type | person']

In [33]:
import xml.etree.ElementTree as ET
from SPARQLWrapper import SPARQLWrapper, JSON

# Initialize the SPARQL endpoint
endpoint_url = "http://dbpedia.org/sparql"
dbpedia = SPARQLWrapper(endpoint_url)

# Parse the XML file
#tree = ET.parse('webnlg-dataset/data/dev/dev.xml')
root = tree.getroot()

# Initialize a list to store the dictionaries
instances = []

# Iterate over each entry
for entry in root.iter('entry'):
    # Get the entity ID
    eid = entry.attrib['eid']
    
    # Initialize a list to store the OTriples
    otriples = []
    
    # Iterate over each otriple in the originaltripleset
    for otriple in entry.find('originaltripleset'):
        # Append the otriple to the list
        otriples.append(otriple.text)
        
    # Concatenate the otriples into a long string
    otriples_str = '-'.join(otriples)
    
    # Get the entity and class information using the function defined earlier
    entities = [ot.split(' | ')[0] for ot in otriples]
    #types = [get_entity_type(e, dbpedia) for e in entities]
    classes = [get_entity_class('http://dbpedia.org/resource/'+e) for e in entities]
  
    
    # Concatenate the entity, type, and class information into a long string
    types_str = '-'.join([f'{e} - type - {c}' for e,  c in zip(entities, classes)])
    print(types_str)
    
    # Create a dictionary for the entry
    entry_dict = {
        'Instance KG': {
            'ID': eid,
            'OTriples': otriples_str,
            'Types KG': types_str
        }
    }
    
    # Append the entry dictionary to the instances list
    instances.append(entry_dict)
    
# Print the instances list
print(instances)


Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person
Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person
Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person
Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person
Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person
Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - type - person-Alan_Bean - 

KeyboardInterrupt: 

In [15]:
a = get_entity_class('http://dbpedia.org/resource/United_States')
print(a)

person
